# Readme First and Second tasks

The first code cell contains the lines of code that enabled the usage of Google Drive files.

The following and most important cell of code, contains the commands and the implementation of the AdaBoost algorithm for face detection. 

1. VideoCapture() is a function with no returned value, which creates the video and specifies its context, such as the type of renderization. For that purpose, in its inside, it was created an async function, which returns a Promise. When the async function returns a value, the Promise will be resolved with that value. The await expression makes it possible to interrupt the execution of the async, and waits for the resolution of the Promise. The purpose of doint this was to apply the Attentional Cascade Algorithm. Then, the face and eyes detection is done using the Cascade Method, which returns rectangles with its detective boundaries.

2. byte2image and image2byte, respectively encode and decode an image to a byte-like object, on the ASCII format.

3. Implementation of the cascade classifiers is due to its huge improvement of time efficiency. Instead of testing all the features available on a specific window, what is done is that these features are divided in collections, which will be focused on fiding the non-face regions, therefore, minimizing the false positive detection.

4. The detect() function makes use of the Cascade Method to identify the rectangles. Its minSize() and maxSize() parameters have the following meaning respectivelly : objects smaller than that are ignored; objects larger than that are ignored. These parameters, together with scaleFactor can improve the time execution, so the algorithm wouldn't lose to much time analysing data that are too large or to small. Since the scaleFactor() specifies how much the image is reduced at each and every image scale, if we have it too small, the time execution will increase.

5. Besides, what was done to reduce the computation time corresponds to search for an exception that reduces the area of interest. Like this, the algorithm doesn't waste time on areas not corresponding to the face. 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Third and Fourth Tasks

1. In this part of the Project, we should be able to improve the detection with the aid of CamShift algorithm. According to its demo version, the mouse is responsible for providing the initial point of the search. 

2. In the current version of the project, it's necessary to replace that correspondent mouse function by automating it with the AdaBoost algorithm provided by the previous solution, i.e, the face detection. 

3. The main idea is to make use of the rectangle, i.e the area of interest determined by the face detection code, as the initial points of CamShift, as the track window.  

So in the current version, the evaluation of the face detection is limited to the first iteration for which a face is detected, that is, the algorithm will run until it finally finds a face. Once it's done, it stops. Now on, the rectangles extracted will be the track window on which the CamShift algorithm will initiate its search. 

4. Then, a histogram computation is performed on the area of interest limited to the masked area, the former being the face area on the figure. This feature is currently used for aiming the computation of the histogram to a unique area.

5. The back projection is done for measuring hoe well some pixels of a given image are to fit in a given histogram. The parameters of the open cv method corresponds to color and pixels ranges. 

6. The stopping criteria of the CamShift was fixed to 10 iterations, or to just one single point of difference regarding the former iteration. Therefore, it's the analog of admiting some error epslon. 

Now, for the fourth task the goal is to erase the face of the image. Ir order to do so, it was necessary to create a condition to extract the probability located in the face area, then setting it to 0. In that way, it was possible to draw the black rectangle. Therefore, the algorithm masks the face in black, enabling the algorithm to identify the hand, since its color map is really similar to the face's. It is convenient to point out that the program may fail if the light conditions are not ideal. Futhermore, it's recommended to try to stand in front of the user camera priorizing your face and whearing long sleeve shirt.

# First to Fourth Task

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode #module used to convert binary or text data into ASCII
import numpy as np
from PIL import Image
import io #module for dealing with several types of I/O
import cv2 #Open CV Library
import time 
from datetime import timedelta
from google.colab.patches import cv2_imshow
import ipywidgets as widgets
import matplotlib.pyplot as plt
import os
from PIL import Image
import csv

In [ ]:
cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml") #face detection
nested = cv2.CascadeClassifier("haarcascade_eye.xml") #eyes detectiond

In [ ]:
# Function to decode byte-like object into image
def byte2image(byte):
  jpeg = b64decode(byte.split(',')[1])
  im = Image.open(io.BytesIO(jpeg))
  return np.array(im)

# Function to do the opposite of byte2image function
def image2byte(image):
  # Creates an image memory created by exporting the array interface from parameter image
  image = Image.fromarray(image)
  # Define a buffer to hold the data, which is kept as bytes
  buffer = io.BytesIO()
  # Save image to the buffer
  image.save(buffer, 'jpeg')

  buffer.seek(0)  #0 : reference point is in the beggining of the file
  x = b64encode(buffer.read()).decode('utf-8')
  return x

In [ ]:
# Function for detecting objects of different sizes based on the input image using the Cascade Method
def detect(img, cascade):
    # The detected objects are returned as a list of rectangles
    # as we update the scale factor by a small factor (0.1~0.2), we can see how time improvement is updated
    rects = cascade.detectMultiScale(img, scaleFactor=1.1, minNeighbors=4, minSize=(30, 30),
                                     flags=cv2.CASCADE_SCALE_IMAGE) #flags parameter has the same meaning for an old cascade
    
    if len(rects) == 0:
        return []
    rects[:,2:] += rects[:,:2]
    return rects

# Drawn rectangles over the image for recongnition 
def draw_rects(img, rects, color):
    #(x1, y1) are starting coordinates of rectangle; (x2, y2) are ending coordinates
    for x1, y1, x2, y2 in rects:
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)


In [ ]:
####################### Face Detection #######################
def VideoCapture():
  #we create a HTML element with div type
  js = Javascript('''
    async function create(){
      div = document.createElement('div');
      document.body.appendChild(div);
      //we create the video element and make it playable inline
      video = document.createElement('video');
      video.setAttribute('playsinline', '');

      div.appendChild(video);

      //getting access to camera
      //video source is facing away from user (i.e to the environment), example : back camera on a smartphone
      //stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "environment"}});  

      //video source is facing toward the user (i.e to the environment), example : front camera on a smartphone
      stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "user"}});

      //here we set the stream to be the source of the media associated to video
      video.srcObject = stream;

      //returns a promise, enabling us to async+await before updating it 
      await video.play();

      canvas =  document.createElement('canvas');
      //setting dimensions of canvas element
      canvas.width = video.videoWidth; 
      canvas.height = video.videoHeight;
      //create a bidimensional renderization object and draw the video into the context
      canvas.getContext('2d').drawImage(video, 0, 0);

      div_out = document.createElement('div');
      document.body.appendChild(div_out);
      img = document.createElement('img');
      div_out.appendChild(img);
    }

    async function capture(){
        return await new Promise(function(resolve, reject){
            pendingResolve = resolve;
            canvas.getContext('2d').drawImage(video, 0, 0); 
            result = canvas.toDataURL('image/jpeg', 0.8);
            pendingResolve(result);
        })
    }

    function showimg(imgb64){
        img.src = "data:image/jpg;base64," + imgb64;
    }

  ''')
  display(js)

# It allows access to computer camera using JavaScript
VideoCapture()
eval_js('create()')

cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml") #face detection
nested = cv2.CascadeClassifier("haarcascade_eye.xml") #eyes detection

####################### DETECT TRACK WINDOW #######################

track_window = [] # Empty list
                  # Variable to store the coordinates of rectangles retrieved from Face Detection

while len(track_window) == 0:
  byte = eval_js('capture()')
  im = byte2image(byte)
  # Convert the original colorspace of im to COLOR_BGR2GRAY
  gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
  track_window = detect(gray, cascade)
  vis = im.copy() # L. 119
  draw_rects(vis, track_window, (0, 255, 0))

x0, y0, x1, y1 = track_window[0]
hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv, np.array((0., 60., 32.)), np.array((180., 255., 255.))) # Area of the face
hsv_roi = hsv[y0:y1, x0:x1]
mask_roi = mask[y0:y1, x0:x1]

# Computing the histogram of hsv_roi for the masked pixels, [0] corresponds to computation in gray-scale
hist = cv2.calcHist( [hsv_roi], [0], mask_roi, [16], [0, 180] )
cv2.normalize(hist, hist, 0, 255, cv2.NORM_MINMAX)

# Stop criteria
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

while True: 
  byte = eval_js('capture()')
  im = byte2image(byte)
  vis = im.copy()

  gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
  mask = cv2.inRange(hsv, np.array((0., 60., 32.)), np.array((180., 255., 255.))) # Area of the face
  hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
  prob = cv2.calcBackProject([hsv], [0], hist, [0, 180], 1)
  prob &= mask

  ####################### Performing Camshift Over Track Window #######################
  rect, track_window = cv2.CamShift(prob, track_window, term_crit)
  x, y, w, h = track_window
  img2 = cv2.rectangle(im, (x, y), (x+w, y+h), 255, 2)

  # Erase the face area 
  prob[y:y+h,x:x+w]=0
  
  # Add another camshift here to detect hand
  byte = eval_js('capture()')
  im = byte2image(byte)
  im_w = im.shape[1] # Image width 
  im_h = im.shape[0] # Image heigth 

  # Here, Camshift is executed with prob on face region put to 0
  track_window_hand = (0, 0, im_w, im_h)
  rect_h, track_window_hand = cv2.CamShift(prob, track_window_hand, term_crit)
  xH, yH, wH, hH = track_window_hand
 
  img2 = cv2.rectangle(prob, (xH,yH), (xH+wH, yH+hH), 255, 2)

  eval_js('showimg("{}")'.format(image2byte(img2)))

<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored

Here we want to create two data sets, respectively dataset16 and dataset224 for 16x16 and 224x224 images. Those images correspond to the hand, selected by the CamShift algorithm performed over the area of interest. 

# Fifth task:

In [ ]:
dataset16 = "/content/gdrive/MyDrive/Computer Vision/dataset16/"
dataset224 = "/content/gdrive/MyDrive/Computer Vision/dataset224/"

In [ ]:
VideoCapture()
eval_js('create()')

byte = eval_js('capture()') 
im = byte2image(byte)
im_w = im.shape[1] # Image width 
im_h = im.shape[0] # Image heigth
tracking_window_hand = (0, 0, im_w, im_h)

################################### Perform the Face Detection ###################################

track_window = [] # Variable to store rectangle coordinates 

while len(track_window)==0:
  byte = eval_js('capture()')
  im = byte2image(byte)
  gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
  ## Face detection
  track_window = detect(gray, cascade)
  vis = im.copy()
  draw_rects(vis, track_window, (0, 255, 0))

################################### Perform CamShift ###################################

x0, y0, x1, y1 = track_window[0] # get the coordinates of rectangle, [0] since we have only one face 

# ROI for CamShift
hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
hsv_roi = hsv[y0:y0+y1, x0:x0+x1]
mask_roi = mask[y0:y0+y1, x0:x0+x1]
hist = cv2.calcHist( [hsv_roi], [0], mask_roi, [16], [0, 180] )
cv2.normalize(hist, hist, 0, 255, cv2.NORM_MINMAX)

# Termination criteria, either 10 iterations or move by at least 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )
print(term_crit)

# n = number of images to store 
n = 1

# Here, still, we perform the CamShift
for k in range(n):
  byte = eval_js('capture()')
  im = byte2image(byte)
  vis = im.copy()
  gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
  mask = cv2.inRange(hsv, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
  hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
  prob = cv2.calcBackProject([hsv],[0],hist,[0,180],1)
  prob &= mask
  # Now performing CamShift to update the coordinates
  ret, track_window = cv2.CamShift(prob, track_window, term_crit)
  # Draw it on image
  x, y, w, h = track_window
  img2 = cv2.rectangle(im, (x,y), (x+w,y+h), 255,2)
 
  # Erase the face area from prob
  prob[0:im_h,x-20:x+w+20] = 0   # Here we put a margin = 20 

  # Hand detection
  res_h, tracking_window_hand = cv2.CamShift(prob, tracking_window_hand, term_crit)
  xh,yh,wh,hh = tracking_window_hand
  img2 = cv2.rectangle(im, (xh,yh), (xh+wh,yh+hh), 255,2)

  hand_crop = img2[0:im_h,0:x-20]
  hand_crop_prob = prob[0:im_h,0:x-20]

  # Store 16x16 and 224x224 images to Drive 
  hand16 = cv2.resize(hand_crop_prob,(16,16))
  hand224 = cv2.resize(hand_crop_prob,(224,224))
  
  path16 = dataset16+'test16.jpg'
  path224 = dataset224+'test224.jpg'
  
  cv2.imwrite(path16, hand16)
  cv2.imwrite(path224, hand224)
  
  eval_js('showimg("{}")'.format(image2byte(prob)))

<IPython.core.display.Javascript object>

(3, 10, 1)


# Sixth Task

In [ ]:
# Input string to get the letter the user wants to store
letter = str(input('\nInsert the letter (uppercase): '))
# Input integer to inform how many samples 
max = int(input('\nInsert the maximum number of pictures you want to stock: ')) 


Insert the letter (uppercase): A

Insert the maximum number of pictures you want to stock: 10


In [ ]:
VideoCapture()
eval_js('create()')

byte = eval_js('capture()') 
im = byte2image(byte)
im_w = im.shape[1] # Image width 
im_h = im.shape[0] # Image heigth
tracking_window_hand = (0, 0, im_w, im_h)

###################################  Loop to detect the face once ################################### 

track_window = [] # New variable where we will store the rectangle's coordinates

while len(track_window)==0:
  byte = eval_js('capture()')
  im = byte2image(byte)
  gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
  ## Face detection
  track_window = detect(gray, cascade)
  vis = im.copy()
  draw_rects(vis, track_window, (0, 255, 0))

################################### CamShift ###################################

x0, y0, x1, y1 = track_window[0] # Get coordinates of rectangle corresponding to face

# Initiate the ROI for CamShift
hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
hsv_roi = hsv[y0:y0+y1, x0:x0+x1]
mask_roi = mask[y0:y0+y1, x0:x0+x1]
hist = cv2.calcHist( [hsv_roi], [0], mask_roi, [16], [0, 180] )
cv2.normalize(hist, hist, 0, 255, cv2.NORM_MINMAX)

# Setup the termination criteria, either 10 iteration or move by atleast 1 pt
term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1 )

#CamShift
for k in range(max):
  byte = eval_js('capture()')
  im = byte2image(byte)
  vis = im.copy()
  gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
  mask = cv2.inRange(hsv, np.array((0., 60., 32.)), np.array((180., 255., 255.)))
  hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
  prob = cv2.calcBackProject([hsv],[0],hist,[0,180],1)
  prob &= mask
  # CamShift update location
  ret, track_window = cv2.CamShift(prob, track_window, term_crit)
  # Draw it on image
  x, y, w, h = track_window
  img2 = cv2.rectangle(im, (x,y), (x+w,y+h), 255,2)
 
  # Erase the face 
  prob[0:im_h,x-20:x+w+20]=0   

  # CamShift to track the hand
  res_h, tracking_window_hand = cv2.CamShift(prob, tracking_window_hand, term_crit)
  xh,yh,wh,hh = tracking_window_hand
  img2 = cv2.rectangle(im, (xh,yh), (xh+wh,yh+hh), 255,2)
 
  hand_crop = img2[0:im_h,0:x-20]
  hand_crop_prob = prob[0:im_h,0:x-20]

  # save 16x16 and 224x224 images to Drive 
  hand16 = cv2.resize(hand_crop_prob, (16,16))
  hand224 = cv2.resize(hand_crop_prob, (224,224))
  
  path16 = dataset16+letter.upper()+'_'+str(k)+'_16.jpg'
  path224 = dataset224+letter.upper()+'_'+str(k)+'_224.jpg'
  
  cv2.imwrite(path16, hand16)
  cv2.imwrite(path224, hand224)
  
  eval_js('showimg("{}")'.format(image2byte(prob)))

<IPython.core.display.Javascript object>

In [ ]:
for dirname, _, filenames in os.walk(dataset16):
  for image in filenames:
    im = Image.open(image)
    im_array = np.asarray(im).ravel().tolist()
    im_array.insert(0,image[0])
    with open('dataset.txt', 'a',newline='') as outfile:
      writer = csv.writer(outfile)
      writer.writerow(im_array)

In [ ]:
lines = open('dataset.txt').readlines()
random.shuffle(lines)
open('dataset.txt', 'w').writelines(lines)
print('\nDone.')

FileNotFoundError: ignored